In [205]:
import pandas as pd
import numpy as np

In [206]:
data = pd.read_csv("../data/train.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [207]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [208]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Feature engineering + data cleaning

In [209]:
keyFactors = [

          # Identifiant (à conserver pour référence)
        'Id',
        
        # Variables structurelles essentielles
        'OverallQual',      # Qualité générale
        'OverallCond',      # Condition générale
        'YearBuilt',        # Année de construction 
        'YearRemodAdd',     # Année de rénovation
        
        # Surfaces
        'GrLivArea',        # Surface habitable
        'TotalBsmtSF',      # Surface totale du sous-sol
        'LotArea',          # Surface du terrain
        
        # Garage et extérieur
        'GarageArea',       # Surface du garage
        'GarageCars',       # Capacité du garage en voitures
        'GarageYrBlt',      # Année de construction du garage
        'GarageType',       # Type de garage
        'GarageFinish',     # Finition du garage
        
        # Pièces et salles de bain
        'FullBath',         # Salles de bain complètes
        'HalfBath',         # Demi-salles de bain
        'BedroomAbvGr',     # Chambres au-dessus du sol
        'KitchenAbvGr',     # Cuisines au-dessus du sol
     
        # Qualité
        'KitchenQual',      # Qualité de la cuisine
        'ExterQual',        # Qualité extérieure
        'ExterCond',        # Condition extérieure
        'BsmtCond',         # Condition du sous-sol
        'HeatingQC',        # Qualité du chauffage
        
        # Localisation
        'Neighborhood',     # Quartier
        'MSZoning',         # Zonage
        
        # Caractéristiques additionnelles de valeur
        'Fireplaces',       # Nombre de cheminées
        'FireplaceQu',      # Qualité des cheminées
        'WoodDeckSF',       # Surface de la terrasse en bois
        'OpenPorchSF',      # Surface du porche ouvert
        'Foundation',       # Type de fondation
        'CentralAir',       # Climatisation centrale
        
        # Variables de vente (pour l'entraînement)

        'SaleType',         # Type de vente
        'SaleCondition',    # Condition de vente
        'MiscFeature',        # Commonditer qui ne figure pas details

]

In [210]:
df = data[keyFactors]
df.set_index("Id", inplace=True)
df.head()

,OverallQual,OverallCond,YearBuilt,YearRemodAdd,GrLivArea,TotalBsmtSF,LotArea,GarageArea,GarageCars,GarageYrBlt,...,MSZoning,Fireplaces,FireplaceQu,WoodDeckSF,OpenPorchSF,Foundation,CentralAir,SaleType,SaleCondition,MiscFeature
Id,,,,,,,,,,,,,,,,,,,,,
1,7,5,2003,2003,1710,856,8450,548,2,2003.0,...,RL,0,NaN,0,61,PConc,Y,WD,Normal,NaN
2,6,8,1976,1976,1262,1262,9600,460,2,1976.0,...,RL,1,TA,298,0,CBlock,Y,WD,Normal,NaN
3,7,5,2001,2002,1786,920,11250,608,2,2001.0,...,RL,1,TA,0,42,PConc,Y,WD,Normal,NaN
4,7,5,1915,1970,1717,756,9550,642,3,1998.0,...,RL,1,Gd,0,35,BrkTil,Y,WD,Abnorml,NaN
5,8,5,2000,2000,2198,1145,14260,836,3,2000.0,...,RL,1,TA,192,84,PConc,Y,WD,Normal,NaN


### Let's add some features !
we have some redundants features storing the same values and sharing the information. Those columns can be merged by selected the second column.

In [211]:
data[['Exterior1st', 'Exterior2nd', 'Condition1', 'Condition2']]

,Exterior1st,Exterior2nd,Condition1,Condition2
0,VinylSd,VinylSd,Norm,Norm
1,MetalSd,MetalSd,Feedr,Norm
2,VinylSd,VinylSd,Norm,Norm
3,Wd Sdng,Wd Shng,Norm,Norm
4,VinylSd,VinylSd,Norm,Norm
...,...,...,...,...
1455,VinylSd,VinylSd,Norm,Norm
1456,Plywood,Plywood,Norm,Norm
1457,CemntBd,CmentBd,Norm,Norm
1458,MetalSd,MetalSd,Norm,Norm


In [221]:
df['Exterior'] = data['Exterior2nd'].values
df['Condition'] = data['Condition2'].values

We can also add the house's lifespan from the build year till the purchase

In [220]:
df["Lifespan"] = np.int64(data["YrSold"] - data["YearBuilt"])
df.fillna({"LifeSpan": 0}, inplace=True) # there is no duration when the result is NA
#####
df = df[df.columns.sort_values()] # sorts the columns in alphabetic order
df.head(3)

,BedroomAbvGr,BsmtCond,CentralAir,Condition,ExterCond,ExterQual,Exterior,FireplaceQu,Fireplaces,Foundation,...,Neighborhood,OpenPorchSF,OverallCond,OverallQual,SaleCondition,SaleType,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
Id,,,,,,,,,,,,,,,,,,,,,
1,3,TA,Y,Norm,TA,Gd,VinylSd,Empty,0,PConc,...,CollgCr,61,5,7,Normal,WD,856,0,2003,2003
2,3,TA,Y,Norm,TA,TA,MetalSd,TA,1,CBlock,...,Veenker,0,8,6,Normal,WD,1262,298,1976,1976
3,3,TA,Y,Norm,TA,Gd,VinylSd,TA,1,PConc,...,CollgCr,42,5,7,Normal,WD,920,0,2001,2002


### Data cleaning

In [214]:
# removing duplicates and checking missing values
df.drop_duplicates(inplace=True)
a = df.isna().sum()
a[a>0]

BsmtCond          37
FireplaceQu      690
GarageFinish      81
GarageType        81
GarageYrBlt       81
MiscFeature     1406
Lifespan           1
dtype: int64

samples having **NA** values means that that feature does not exist for that house. these values won't be dropped but will be replaced by "empty" and encoded during the process.

In [215]:
## replaces the NA with "Empty"
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    temp = df.copy()
    cols_with_empty_values = a[a>0].index
    for c in cols_with_empty_values:
        temp[c] = temp[c].fillna("Empty")
    return temp

df = fill_missing_values(df)

In [224]:
# let's check values type for each sample
def check(df: pd.DataFrame):
    temp = df.dropna(axis=0)
    print("start cheking ...")
    for col in temp.columns:
        if temp[col].dtype == np.int64:
            try:
                np.int64(temp[col])
            except Exception as e:
                print(f"'{col.capitalize()}' feature should have int64 type for all samples")
        elif temp[col].dtype == np.float64:
            try:
                np.float64(temp[col])
            except Exception as e:
                print(f"'{col.capitalize()}' feature should have float64 type for all samples")
        else :
            try:
                np.object_(temp[col])
            except Exception as e:
                print(f"'{col.capitalize()}' feature should have object type for all samples")
    print("All columns are checked ")

check(df)
print(f"we have {df.shape[0]} samples and {df.shape[1]} features with the houses id set as index")

start cheking ...
All columns are checked 
we have 1460 samples and 35 features with the houses id set as index


## Data exploration
we will display the insights and highlight how the selected features are relevant for the sale price prediction.
features are categorized into nine(9) parts:
* **Essential structural variables** : *'OverallQual'*, *'OverallCond'*, *'YearBuilt'*, *'YearRemodAdd'*
* **Surfaces** : *GrLivArea*, *TotalBsmtSF*, *LotArea*  
* **Garage and Exterior** : *GarageArea*, *GarageCars*, *GarageYrBlt*, *GarageType*, *GarageFinish*  
* **Rooms and Bathrooms** : *FullBath*, *HalfBath*, *BedroomAbvGr*, *KitchenAbvGr*  
* **Quality** : *KitchenQual*, *ExterQual*, *ExterCond*, *BsmtCond*, *HeatingQC*  
* **Location** : *Neighborhood*, *MSZoning*  
* **Additional Value Features** : *Fireplaces*, *FireplaceQu*, *WoodDeckSF*, *OpenPorchSF*, *Foundation*, *CentralAir*  
* **Sales Variables (for training)** : *SaleType*, *SaleCondition*, *MiscFeature*  
* **Created features** : *Exterior*, *LifeSpan*, *Condition*  